In [21]:
import copy

from alns import ALNS
from alns.select import RouletteWheel
from alns.stop import MaxRuntime
from alns.accept import RecordToRecordTravel

import numpy as np
from functions.part1 import part1
from classes.deliveryrouting import DeliveryRouting

In [22]:
instance_dir = "data/0o50t75s1p100"
dr = DeliveryRouting(instance_dir)
dr.driver_code()

In [23]:
def random_destroy(State : DeliveryRouting, rnd_state : np.random.seed = np.random.seed(42)):
    destroyed = copy.deepcopy(State)

    order_destroyed = np.random.choice(destroyed.orders)
    destroyed.unassigned = order_destroyed


    for time, routes in destroyed.final_result.items():
        for route in routes:
            for bundle in route:
                if order_destroyed in bundle.bundle:
                    bundle.bundle.remove(order_destroyed)
                # if len(bundle.bundle) == 0:
                #     route.remove(bundle)
    return destroyed

In [49]:
def greedy_assign(State: DeliveryRouting, rnd_state : np.random.seed = np.random.seed(42)):
    repaired = copy.deepcopy(State)
    destroyed_order = repaired.unassigned

    min_time = 1e9
    done = False
    for time, routes in repaired.final_result.items():
        for route in routes:
            for bundle in route:
                if destroyed_order.ready_time >= time:
                    if len(bundle.bundle) > 0:
                        if destroyed_order.restaurant_id == bundle.bundle[-1].restaurant_id:
                            if bundle.get_route_cost(repaired.meters_per_minute, repaired.locations) < min_time:
                                min_time = bundle.get_route_cost(repaired.meters_per_minute, repaired.locations)
                                best_route = bundle
                                done = True
    if not done:
        for time, routes in repaired.final_result.items():
            for route in routes:
                for bundle in route:
                    if destroyed_order.ready_time >= time:
                        if bundle.get_route_cost(repaired.meters_per_minute, repaired.locations) < min_time:
                            min_time = bundle.get_route_cost(repaired.meters_per_minute, repaired.locations)
                            best_route = bundle

    print(destroyed_order.id)
    best_route.bundle.append(destroyed_order)
    return repaired


In [50]:
import numpy.random as rnd

alns = ALNS(rnd.RandomState(42))
alns.add_destroy_operator(random_destroy)
alns.add_repair_operator(greedy_assign)

In [51]:
init = dr
select = RouletteWheel([25, 5, 1, 0], 0.8, 1, 1)
accept = RecordToRecordTravel.autofit(init.objective(), 0.02, 0, 9000)
stop = MaxRuntime(10)

result = alns.iterate(init, select, accept, stop)

o191
o33
o182
o111
o195
o163
o41
o117
o191
o208
o61
o65
o166
o54
o178
o203
o189
o192
o235
o216
o233
o146
o10
o178
o84
o240
o132
o215
o44
o40
o117
o243
o55
o150
o118
o84
o179
o142
o69
o151
o251
o7
o40
o59
o111
o42
o42
o29
o42
o144
o196
o148
o91
o156
o96
o216
o78
o144
o22
o117
o164
o249
o114


KeyboardInterrupt: 